## Import modules

In [32]:
import pandas as pd
import sqlite3
from faker import Faker

# Dataset does not have technician names or store locations, so we'll make these up

fake = Faker()

## Create database file

In [14]:
conn = sqlite3.connect('database.db')

#Not creating the tables here because pandas can do this for us

## Import csv file with data and check data types

In [15]:
df = pd.read_csv('Copy of Repair Data_v3.csv',
                 index_col=None,
                 parse_dates=['repair_date','repair_start_time','repair_complete_time'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65023 entries, 0 to 65022
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   store_id                    65023 non-null  object        
 1   work_order                  65023 non-null  int64         
 2   repair_date                 65023 non-null  datetime64[ns]
 3   warranty                    65023 non-null  object        
 4   warranty_status             65023 non-null  object        
 5   device_generation           65023 non-null  object        
 6   product_description         65023 non-null  object        
 7   customer_reported_code      50858 non-null  object        
 8   triage_failure_code         39211 non-null  object        
 9   triage_failure_description  39211 non-null  object        
 10  asp repair_code             32216 non-null  object        
 11  asp repair_description      32216 non-null  object    

## Why is store_id an object and not an integer?

In [16]:
df['store_id'].unique()

array(['539', '746', '554', '330', '189', '542', '752', '479', '670',
       '687', '227', '5', '6', '185', '532', '827', '375', '555', '815',
       '304', '198', '274', '800', '861', '468', '403', '11', '558',
       '517', '780', '896', '581', '224', '179', '771', '545', '285',
       '617', '480', '765', '688', '16', '803', '834', '500', '778', '17',
       '18', '268', '392', '212', '19', '21', '610', '22', '718', '363',
       '664', '449', '24', '25', '26', '627', '393', '577', '297', '585',
       '329', '777', '228', '309', '427', '28', '436', '30', '604', '320',
       '336', '486', '892', '31', '559', '783', '32', '702', '230', '36',
       '913', '415', '745', '289', '697', '40', '848', '541', '600',
       '452', '405', '41', '42', '362', '537', '44', '881', '288', '46',
       '562', '812', '788', '49', '796', '50', '440', '51', '769', '547',
       '54', '843', '220', '335', '492', '312', '868', '55', '629', '732',
       '805', '490', '401', '313', '321', '737', '292', 

## Create new column for Depots

In [17]:
df['depot'] = df['store_id'].str.contains('-Depot').astype(bool)
df.head()

,store_id,work_order,repair_date,warranty,warranty_status,device_generation,product_description,customer_reported_code,triage_failure_code,triage_failure_description,...,repair_start_time,repair_complete_time,repair_location,diag_code,last_rfp_status,technician_id,parts_problems,rework_90_day,sales,depot
0,539,12984423,2021-08-10,N,EW,Gen 3,Smartphone 3,NaN,NaN,NaN,...,2021-08-07 10:00:00,2021-08-10 08:14:00,STORE,NaN,Repaired - RFP,11991,parts issue,0,253.64,False
1,539,12742816,2021-08-10,N,EW,Gen 4,Smartphone 4 XL,NaN,NaN,NaN,...,2021-07-15 12:02:00,2021-08-10 08:12:00,STORE,NaN,Repaired - RFP,11991,parts issue,0,265.25,False
2,746,13008982,2021-08-09,N,EW,Gen 3,Smartphone 3 XL,NaN,NaN,NaN,...,2021-08-09 12:57:00,2021-08-09 16:39:00,STORE,NaN,Repaired - RFP,10681,parts issue,0,178.86,False
3,554,13009475,2021-08-09,N,OOW,Gen 3a,Smartphone 3a,Physical Damage,T028,"Display dead pixel, dark spots or foreign mate...",...,2021-08-09 15:16:00,2021-08-09 18:04:00,STORE,NaN,Repaired - RFP,11745,parts issue,0,109.99,False
4,554,12878951,2021-08-09,N,OOW,Gen 1,Smartphone,Power,NaN,NaN,...,2021-07-28 10:46:00,2021-08-09 12:03:00,STORE,NaN,Other - RFP,11975,--,0,0.00,False


## Remove '-Depot' from the store_id so it can be stored as integer

In [18]:
df['store_id'] = df['store_id'].str.replace('-Depot','').astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65023 entries, 0 to 65022
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   store_id                    65023 non-null  int64         
 1   work_order                  65023 non-null  int64         
 2   repair_date                 65023 non-null  datetime64[ns]
 3   warranty                    65023 non-null  object        
 4   warranty_status             65023 non-null  object        
 5   device_generation           65023 non-null  object        
 6   product_description         65023 non-null  object        
 7   customer_reported_code      50858 non-null  object        
 8   triage_failure_code         39211 non-null  object        
 9   triage_failure_description  39211 non-null  object        
 10  asp repair_code             32216 non-null  object        
 11  asp repair_description      32216 non-null  object    

## Set boolean types to save space

In [19]:
df['same_day_completion'] = df['same_day_completion'].astype(bool)
df['rework_90_day'] = df['rework_90_day'].astype(bool)
df['warranty'] = df['warranty'].str.replace('N','0').replace('Y','1').astype(bool)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65023 entries, 0 to 65022
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   store_id                    65023 non-null  int64         
 1   work_order                  65023 non-null  int64         
 2   repair_date                 65023 non-null  datetime64[ns]
 3   warranty                    65023 non-null  bool          
 4   warranty_status             65023 non-null  object        
 5   device_generation           65023 non-null  object        
 6   product_description         65023 non-null  object        
 7   customer_reported_code      50858 non-null  object        
 8   triage_failure_code         39211 non-null  object        
 9   triage_failure_description  39211 non-null  object        
 10  asp repair_code             32216 non-null  object        
 11  asp repair_description      32216 non-null  object    

## Break up data into different tables

In [34]:
# Create a table of the unique store_id and depot

store_table = df[['store_id','depot']]\
    .sort_values('store_id')\
    .drop_duplicates()\
    .set_index('store_id')

# Set up faker to create our fake addresses

store_locations = [(fake.street_address(),
                    fake.city(),
                    fake.state_abbr(),
                    fake.zipcode())
                   for _ in range(len(store_table))]

# Create a dictionary to store the fake data

address_dict = {}
for i, store in enumerate(store_table.index):
    address_dict[store] = store_locations[i % len(store_locations)]

# Add the fake addresses to the dataframe

store_table['address'] = [address_dict[store][0] for store in store_table.index]
store_table['city'] = [address_dict[store][1] for store in store_table.index]
store_table['state'] = [address_dict[store][2] for store in store_table.index]
store_table['zipcode'] = [address_dict[store][3] for store in store_table.index]

store_table.head()

,depot,address,city,state,zipcode
store_id,,,,,
1,False,08248 Gardner Row,Port Sharonstad,FM,14953
2,False,4911 Smith Plains Suite 090,Longhaven,AZ,49105
3,False,39617 Orr Union Suite 896,South Christopherbury,SC,98252
4,False,22471 Paul Ridges,Williamschester,MH,72143
5,False,45019 Teresa Lakes,North Brandonton,RI,10159


In [21]:
store_table.to_sql('store', conn, if_exists="replace")

731

In [22]:
work_order_table = df.drop(columns=['same_day_completion','depot']).set_index('work_order')
work_order_table.head()

,store_id,repair_date,warranty,warranty_status,device_generation,product_description,customer_reported_code,triage_failure_code,triage_failure_description,asp repair_code,...,rma_country,repair_start_time,repair_complete_time,repair_location,diag_code,last_rfp_status,technician_id,parts_problems,rework_90_day,sales
work_order,,,,,,,,,,,,,,,,,,,,,
12984423,539,2021-08-10,True,EW,Gen 3,Smartphone 3,NaN,NaN,NaN,NaN,...,US,2021-08-07 10:00:00,2021-08-10 08:14:00,STORE,NaN,Repaired - RFP,11991,parts issue,False,253.64
12742816,539,2021-08-10,True,EW,Gen 4,Smartphone 4 XL,NaN,NaN,NaN,NaN,...,US,2021-07-15 12:02:00,2021-08-10 08:12:00,STORE,NaN,Repaired - RFP,11991,parts issue,False,265.25
13008982,746,2021-08-09,True,EW,Gen 3,Smartphone 3 XL,NaN,NaN,NaN,NaN,...,CA,2021-08-09 12:57:00,2021-08-09 16:39:00,STORE,NaN,Repaired - RFP,10681,parts issue,False,178.86
13009475,554,2021-08-09,True,OOW,Gen 3a,Smartphone 3a,Physical Damage,T028,"Display dead pixel, dark spots or foreign mate...",R17,...,US,2021-08-09 15:16:00,2021-08-09 18:04:00,STORE,NaN,Repaired - RFP,11745,parts issue,False,109.99
12878951,554,2021-08-09,True,OOW,Gen 1,Smartphone,Power,NaN,NaN,NaN,...,US,2021-07-28 10:46:00,2021-08-09 12:03:00,STORE,NaN,Other - RFP,11975,--,False,0.00


In [23]:
work_order_table.to_sql('work_order', conn, if_exists="replace")

65023

In [38]:
#Get a unique list of tech_id and convert to dataframe
#drop_duplicates on a single column creates a series, not dataframe
#to_frame fixes this, then set index to tech id

tech_table = df['technician_id'].drop_duplicates().to_frame().set_index('technician_id')

#Create fake first and last names for each tech

tech_table['first_name'] = [fake.first_name() for _ in range(len(tech_table))]
tech_table['last_name'] = [fake.last_name() for _ in range(len(tech_table))]
tech_table['full_name'] = tech_table['first_name'] + ' ' + tech_table['last_name']

tech_table.head()

,first_name,last_name,full_name
technician_id,,,
11991,Anna,Morris,Anna Morris
10681,Elizabeth,Forbes,Elizabeth Forbes
11745,Sandra,Hall,Sandra Hall
11975,Crystal,Gross,Crystal Gross
10513,Sarah,Wilson,Sarah Wilson


In [35]:
tech_table.to_sql('technician', conn, if_exists="replace")

3634

## That's enough tables for now. I could create a table for devices too, but it is not necessary.

In [39]:
# Close the database connection since we're done with it for now.

conn.close()